## 大衍求一术

大衍问题源于《孙子算经》中的“物不知数”问题：
“今有物，不知其数，三三数之剩二，五五数之剩三，七七数之剩二，问物几何？”

参考：

https://baike.baidu.com/item/%E5%A4%A7%E8%A1%8D%E6%B1%82%E4%B8%80%E6%9C%AF/5523066

In [3]:
def gcd(a, b):
    if a == 0:
        return b
    if a < b:
        return gcd(b % a, a)
    else:
        return gcd(b, a)
gcd(101, 1001), gcd(101, 10001), gcd(1001, 10001)

(1, 1, 1)

We want to use this example to show you the process of gradual refinement in algorithm design:

We will start with a crude algorithm, then improve it bit by bit, and finally reach something blazing fast.

In this process, we may develop some useful mathematical theorems, but it is quite fun.

Let us start with simple version.

In [1]:
for x in range(1, 200):
    if (x % 3 == 2) and (x % 5 == 3) and (x % 7 == 2):
        print("found x", x)
        break

found x 23


## Generalizing the problem

We wish to find x such that
* `x % d1 == m1`  (In the original problem, d1 == 3, and m1 == 2)
* `x % d2 == m2`  (In the original problem, d2 == 5, and m2 == 3)
* `x % d3 == m3`  (In the original problem, d3 == 7, and m3 == 2)

### Analyzing the problem

We can borrow our solution for the special case.  Here is one question:

#### How many x's do we have to go through?

We don't want to miss a solution, but we don't want to check too many integers neither.
After a little thought, we can conclude

    If $x$ is a solution, so is `y = x + d1 d2 d3`.

    (You can check for yourself `y % d1 = x % d1`, and the same for `d2` and `d3`)

This fact is very useful.
It means that we only need to search `0, 1, ..., d1 d2 d3 - 1`, but no more.

So our solution can be summarized as the following function:

In [2]:
def remainder_solver_3var(d1, m1, d2, m2, d3, m3):
    ''' return x such that
        x % d1 == m1
        x % d2 == m2
        x % d3 == m3
    '''
    
    for x in range(d1*d2*d3):
        if (x % d1 == m1) and (x % d2 == m2) and (x % d3 == m3):
            return x
    return -1

remainder_solver_3var(3, 2, 5, 3, 7, 2)

23

On average this algorithm will search $d_1 d_2 d_3 / 2$ times before it hits the right answer.

It is not bad, but can we do better?

For this, let us consider some simpler problems.

### One-variable problem

First, let us try the one variable problem:

    Find $x$ such that `x % d1 == m1`.

Well, the answer is just `m1`.

### Two-variable problem

Now, let's try two variables

    Find $x$ such that `x % d1 == m1` and `x % d2 == m2`

Or, with concrete numbers

    Find $x$ such that `x % 3 == 2` and `x % 5 == 3`

Well this isn't too hard neither, we can borrow our solution for three variables.

In [3]:
def remainder_solver_2var(d1, m1, d2, m2):
    ''' return x such that
        x % d1 == m1
        x % d2 == m2
    '''
    
    for x in range(d1*d2):
        if (x % d1 == m1) and (x % d2 == m2):
            return x
    return -1

remainder_solver_2var(3, 2, 5, 3)

8

## Smarter way of solving the two-variable problem

On average, the loop will go over `d1*d2/2` times before it hits the right answer.

But we can certainly do a bit better.

We can start with `m1`, because it is the first number that satisfy `x % d1 == m1`.

The next number to try would be `m1 + d1`, for it is next number that satisfy `x % d1 == m1`.

And the next would be `m1 + d1*2`, the next `m1 + d1*3`

...

In this way, we can dramatically reduce the numbers to search.

Also note that in the searching process, we can be sure that the first condition `x % d1 == m1` is always satisfied.  So in the loop, we only need to check the second condition.


In [4]:
def remainder_solver_2var_v2(d1, m1, d2, m2):
    ''' return x such that
        x % d1 == m1
        x % d2 == m2
    '''
    
    for x in range(m1, d1*d2, d1):
        if x % d2 == m2:
            return x
    return -1

remainder_solver_2var_v2(3, 2, 5, 3)

8

## An even smarter way of solving the two-variable problem

This loop on average will go through `d2/2` numbers, which is much better than the `d1*d2/2` in the previous example.

But we can still do a bit better.

If `d1 < d2` we can swap `(d1, m1)` and `(d2, m2)`, then the loop will go through `d1/2` times.

In [5]:
def remainder_solver_2var_v3(d1, m1, d2, m2):
    ''' return x such that
        x % d1 == m1
        x % d2 == m2
    '''
    
    if d1 < d2: # swap (d1, m1) and (d2, m2)
        d1, m1, d2, m2 = d2, m2, d1, m1
    
    for x in range(m1, d1*d2, d1):
        if x % d2 == m2:
            return x
    return -1

remainder_solver_2var_v3(3, 2, 5, 3)

8

## Three-variable problem

Now let us go back to the three-variable problem.

Here is the brilliant idea.

If we have solved the two-variable problem for `(d1, m1)` and `(d2, m2)` and the solution is `x1_2`, the solution can be reduced a new condition that

    `x % (d1*d2) == x1_2`
    
Then the three-variable problem is reduced to a new two variable problem of

    `x % (d1*d2) == x1_2`
    `x % d3 == m3`
    
So, the three-variable problem is reduced two two-variable problems!

In [6]:
def remainder_solver_3var_v2(d1, m1, d2, m2, d3, m3):
    ''' return x such that
        x % d1 == m1
        x % d2 == m2
        x % d3 == m3
    '''
    
    x1_2 = remainder_solver_2var_v3(d1, m1, d2, m2)
    
    return remainder_solver_2var_v3(d1*d2, x1_2, d3, m3)

remainder_solver_3var_v2(3, 2, 5, 3, 7, 2)

23

## Improvement on the solution of the three-variable problem

We can actually do a bit better.

The average number of loops is `min(d1, d2)/2 + d3/2`.

But if `d3` is the largest of the three, we can swap `(d2, m2)` and `(d3, m3)`, and the number will be reduced to
`(d1 + d2)/2`

In [7]:
def remainder_solver_3var_v3(d1, m1, d2, m2, d3, m3):
    ''' return x such that
        x % d1 == m1
        x % d2 == m2
        x % d3 == m3
    '''
    dmax = max(d1, d2, d3)
    if dmax == d3: # swap d2 and d3
        d2, m2, d3, m3 = d3, m3, d2, m2

    x1_2 = remainder_solver_2var_v3(d1, m1, d2, m2)
    
    return remainder_solver_2var_v3(d1*d2, x1_2, d3, m3)

remainder_solver_3var_v3(3, 2, 5, 3, 7, 2)

23

## Performance test

Let us switch to some large `d1, d2, d3` and see how well the algorithms do.

In [8]:
%timeit remainder_solver_3var(11, 2, 101, 3, 10001, 5)

228 ms ± 3.11 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%timeit remainder_solver_3var_v2(11, 2, 101, 3, 10001, 5)

24.4 µs ± 530 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [10]:
%timeit remainder_solver_3var_v3(11, 2, 101, 3, 10001, 5)

3.02 µs ± 192 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Clearly `v3` is the fastest algorithm, about $10^5$ faster than the original

## Performance test: a very hard problem

In [11]:
remainder_solver_3var(101, 2, 1001, 3, 10001, 5)

719681966

In [12]:
remainder_solver_3var_v2(101, 2, 1001, 3, 10001, 5)

719681966

In [13]:
remainder_solver_3var_v2(101, 2, 10001, 5, 1001, 3)

719681966

## 4-variable problem

Homework